In [2]:
import math
import random
import pygame
import sys
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops

# Size of canvas
width, height = 1000, 750
boidneigbor, numBoids, visualRange = 3, 100, 75

boids = []

def initBoids():
    global boids
    boids = []
    for i in range(numBoids):
        boids.append({
            'x': random.random() * width,
            'y': random.random() * height,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
        })

def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)

def sizeCanvas():
    global width, height
    size = (width, height)
    return pygame.display.set_mode(size)

def flyTowardsCenter(boid):
    centeringFactor = 0.005
    centerX, centerY, numNeighbors = 0, 0, 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors
        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX, moveY = 0, 0

    for otherBoid in boids:
        if otherBoid != boid and distance(boid, otherBoid) < minDistance:
            moveX += boid['x'] - otherBoid['x']
            moveY += boid['y'] - otherBoid['y']

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05
    avgDX, avgDY, numNeighbors = 0, 0, 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1

    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors
        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor

def limitSpeed(boid):
    speedLimit = 15
    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def drawBoid(screen, boid):
    angle = math.atan2(boid['dy'], boid['dx'])
    boid_surface = pygame.Surface((30, 10), pygame.SRCALPHA)
    pygame.draw.polygon(boid_surface, (85, 140, 244), [(0, 0), (0, 10), (-15, 5)])
    rotated_boid = pygame.transform.rotate(boid_surface, math.degrees(angle))
    rotated_rect = rotated_boid.get_rect(center=(boid['x'], boid['y']))
    screen.blit(rotated_boid, rotated_rect)

    if DRAW_TRAIL:
        for point in boid['history']:
            pygame.draw.circle(screen, (85, 140, 244, 102), (int(point[0]), int(point[1])), 1)

# Main animation loop
def animationLoop():
    global boids
    for boid in boids:
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)

        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-50:]

    screen.fill((255, 255, 255))
    for boid in boids:
        drawBoid(screen, boid)

    pygame.display.flip()
    pygame.time.Clock().tick(60)

pygame.init()
screen = sizeCanvas()
DRAW_TRAIL = False
num_time_steps = 100
num_sims = 10
velocities, positions = [], []

for k in range(num_sims):
    initBoids()
    for _ in range(num_time_steps):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        animationLoop()

    time_list = []

    for i in range(len(boids[1]["history"])):
        boid_dict = {}
        if i == 0:
            continue
        else:
            for j in range(len(boids)):
                dic = {}
                dic['x'] = boids[j]['history'][i][0]
                dic['y'] = boids[j]['history'][i][1]
                dic['dx'] = boids[j]['history'][i][0] - boids[j]['history'][i-1][0]
                dic['dy'] = boids[j]['history'][i][1] - boids[j]['history'][i-1][1]
                boid_dict[f'boid{j}'] = dic
        time_list.append(boid_dict)

    with open(f"../data/myjson/mydata{k}.json", "w") as final:
        json.dump(time_list, final)

# Load data for training (example for one simulation)
with open(f"../data/myjson/mydata0.json", "r") as f:
    time_list = json.load(f)

# Convert data to graph format
def create_graph_data(time_list):
    node_features = []
    edge_index = [[], []]
    for time_step in time_list:
        for boid_id, boid_data in time_step.items():
            node_features.append([boid_data['x'], boid_data['y'], boid_data['dx'], boid_data['dy']])
            for other_boid_id, other_boid_data in time_step.items():
                if boid_id != other_boid_id:
                    edge_index[0].append(int(boid_id.replace('boid', '')))
                    edge_index[1].append(int(other_boid_id.replace('boid', '')))
    node_features = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    return Data(x=node_features, edge_index=edge_index)

data = create_graph_data(time_list)

# Define the encoder, decoder, and GRNN models
class Encoder(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(Encoder, self).__init__()
        self.linear = nn.Linear(in_channels, hidden_channels)

    def forward(self, x):
        return torch.relu(self.linear(x))

class Decoder(nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(hidden_channels, out_channels)

    def forward(self, x):
        return self.linear(x)

class RecurrentGNN(MessagePassing):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(RecurrentGNN, self).__init__(aggr='add')
        self.gru = nn.GRUCell(hidden_channels, hidden_channels)
        self.encoder = Encoder(in_channels, hidden_channels)
        self.decoder = Decoder(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.encoder(x)
        out = x
        for _ in range(3):  # Number of recurrent steps
            out = self.propagate(edge_index, x=out)
        out = self.decoder(out)
        return out

    def message(self, x_j):
        return x_j

    def update(self, aggr_out, x):
        return self.gru(aggr_out, x)

# Initialize the model, optimizer, and loss function
model = RecurrentGNN(in_channels=4, hidden_channels=8, out_channels=4)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Training loop
for epoch in range(num_epochs):
    for t in range(len(features_list) - 1):
        real_features = features_list[t]
        next_real_features = features_list[t + 1]
        edge_index = adjacency_matrices[t].nonzero(as_tuple=False).t().contiguous()

        # Train Discriminator
        d_optimizer.zero_grad()
        
        real_labels = torch.ones(next_real_features.size(0), 1)
        fake_labels = torch.zeros(next_real_features.size(0), 1)
        
        real_output = discriminator(next_real_features)
        d_loss_real = criterion(real_output, real_labels)
        
        fake_features = generator(real_features, edge_index)
        fake_output = discriminator(fake_features.detach())
        d_loss_fake = criterion(fake_output, fake_labels)
        
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        g_optimizer.zero_grad()
        
        fake_output = discriminator(fake_features)
        g_loss = criterion(fake_output, real_labels)
        
        g_loss.backward()
        g_optimizer.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{t+1}/{len(features_list)-1}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

# Plotting generator predictions
with torch.no_grad():
    for t in range(len(features_list) - 1):
        real_features = features_list[t]
        edge_index = adjacency_matrices[t].nonzero(as_tuple=False).t().contiguous()
        predicted_features = generator(real_features, edge_index).numpy()
        
        plt.figure(figsize=(10, 6))
        plt.scatter(real_features[:, 0], real_features[:, 1], c='blue', label='Real')
        plt.scatter(predicted_features[:, 0], predicted_features[:, 1], c='red', label='Predicted')
        plt.legend()
        plt.title(f'Timestep {t}')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()



FileNotFoundError: [Errno 2] No such file or directory: '../data/myjson/mydata0.json'